In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import mxnet as mx
from gluonts.dataset.pandas import PandasDataset
from gluonts.mx import DeepAREstimator, Trainer
# from gluonts.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.distribution.neg_binomial import NegativeBinomialOutput
from gluonts.mx.trainer.callback import TrainingHistory

from gluonts.dataset.util import to_pandas
from gluonts.dataset.pandas import PandasDataset
from gluonts.evaluation.backtest import make_evaluation_predictions

In [2]:
mx.context.num_gpus()

0

In [3]:
# df = pd.read_csv('Merged-update_hourly.csv')
df = pd.read_csv('../../../data/pollution.csv')

In [4]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0


In [5]:
df.loc[:, 'timestamp'] = pd.to_datetime(df.loc[:, 'date'], infer_datetime_format=True)

In [6]:
df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
0,2010-01-02 00:00:00,129.0,-16,-4.0,1020.0,1.79,0,0,0,0,1,0,2010-01-02 00:00:00
1,2010-01-02 01:00:00,148.0,-15,-4.0,1020.0,2.68,0,0,0,0,1,0,2010-01-02 01:00:00
2,2010-01-02 02:00:00,159.0,-11,-5.0,1021.0,3.57,0,0,0,0,1,0,2010-01-02 02:00:00
3,2010-01-02 03:00:00,181.0,-7,-5.0,1022.0,5.36,1,0,0,0,1,0,2010-01-02 03:00:00
4,2010-01-02 04:00:00,138.0,-7,-5.0,1022.0,6.25,2,0,0,0,1,0,2010-01-02 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [7]:
df.columns

Index(['date', 'pollution', 'dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain',
       'NE', 'NW', 'SE', 'cv', 'timestamp'],
      dtype='object')

In [8]:
train_df = df.iloc[:int(len(df)*0.8), :]
test_df = df.iloc[int(len(df)*0.8):, :]

In [9]:
train_df.shape

(35040, 13)

In [10]:
test_df

,date,pollution,dew,temp,press,wnd_spd,snow,rain,NE,NW,SE,cv,timestamp
35040,2014-01-01 00:00:00,24.0,-20,7.0,1014.0,143.48,0,0,0,1,0,0,2014-01-01 00:00:00
35041,2014-01-01 01:00:00,53.0,-20,7.0,1013.0,147.50,0,0,0,1,0,0,2014-01-01 01:00:00
35042,2014-01-01 02:00:00,65.0,-20,6.0,1013.0,151.52,0,0,0,1,0,0,2014-01-01 02:00:00
35043,2014-01-01 03:00:00,70.0,-20,6.0,1013.0,153.31,0,0,0,1,0,0,2014-01-01 03:00:00
35044,2014-01-01 04:00:00,79.0,-18,3.0,1012.0,0.89,0,0,0,0,0,1,2014-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43795,2014-12-31 19:00:00,8.0,-23,-2.0,1034.0,231.97,0,0,0,1,0,0,2014-12-31 19:00:00
43796,2014-12-31 20:00:00,10.0,-22,-3.0,1034.0,237.78,0,0,0,1,0,0,2014-12-31 20:00:00
43797,2014-12-31 21:00:00,10.0,-22,-3.0,1034.0,242.70,0,0,0,1,0,0,2014-12-31 21:00:00
43798,2014-12-31 22:00:00,8.0,-22,-4.0,1034.0,246.72,0,0,0,1,0,0,2014-12-31 22:00:00


In [11]:
train_ds = PandasDataset(
    train_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

In [12]:
test_ds = PandasDataset(
    test_df,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
    freq = 'H'
)

#### training

In [13]:
K = 6
n_hours = 24*3

In [14]:
trainer=Trainer(
    learning_rate=1e-3,
    epochs=200,
    num_batches_per_epoch=100,
    add_default_callbacks=True,
    weight_decay=1e-08,)

estimator = DeepAREstimator(
    prediction_length=K,
    freq="H",
    distr_output = NegativeBinomialOutput(),
    use_feat_dynamic_real=True,
    batch_size=128,
    trainer=trainer,
)

model = estimator.train(train_ds)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:35<00:00,  3.35s/it, epoch=76/200, avg_epoch_loss=3.96]


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:49<00:00,  3.49s/it, epoch=152/200, avg_epoch_loss=3.9]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [05:47<00:00,  3.47s/it, epoch=200/200, avg_epoch_loss=3.91]


#### predicted y

In [15]:
# forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=500)
# forecasts = list(forecast_it)
# tss = list(ts_it)
# evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
# agg_metrics, item_metrics = evaluator(tss, forecasts)

In [16]:
quantiles = ['0.025', '0.1', '0.2', '0.3', '0.4', '0.5', '0.6', '0.7', '0.8', '0.9', '0.975']
q50 = 5

In [17]:
test_ds_predict_q50_all = np.array([])

for i in range(len(test_df)-n_hours-K+1):
    print(i)
    
    cut = df.iloc[len(train_df)+i:len(train_df)+i+n_hours+K, :]

    test_ds = PandasDataset(
    cut,
    target = 'pollution',
    timestamp = 'timestamp',
    feat_dynamic_real = ['dew', 'temp', 'press', 'wnd_spd', 'snow', 'rain', 'NE', 'NW', 'SE', 'cv',],
        freq = 'H'
    )
    
    forecast_it, ts_it = make_evaluation_predictions(dataset=test_ds, predictor=model, num_samples=200)
    test_ds_predict = list(forecast_it)
    test_ds_predict_quantiles = test_ds_predict[0].samples
    test_ds_predict_q50 = test_ds_predict_quantiles.mean(axis=0)
    test_ds_predict_q50_all = np.concatenate((test_ds_predict_q50_all, test_ds_predict_q50), axis=None)
    print(test_ds_predict_q50_all)
    print(test_ds_predict_q50_all.shape)
    print('------')

0
[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275]
(6,)
------
1
[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817]
(12,)
------
2
[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817  87.34500122  89.20999908  92.5
  91.31999969  92.10500336  92.51999664]
(18,)
------
3
[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817  87.34500122  89.20999908  92.5
  91.31999969  92.10500336  92.51999664  92.48500061  92.95999908
  95.31999969  97.08999634  97.25499725  97.34999847]
(24,)
------
4
[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499

[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817  87.34500122  89.20999908  92.5
  91.31999969  92.10500336  92.51999664  92.48500061  92.95999908
  95.31999969  97.08999634  97.25499725  97.34999847  82.31500244
  82.91500092  86.77999878  90.76499939  92.23500061  92.14499664
  82.36499786  85.48999786  90.90499878  94.80999756  94.59999847
  93.84500122  81.62000275  83.00499725  86.51999664  86.95500183
  86.04499817  84.94499969  87.66000366  88.09999847  90.43499756
  90.36000061  90.06999969  86.05500031  90.49500275  93.45500183
  94.73500061  96.25        96.12999725  94.37000275  98.11000061
  98.76499939 101.23500061 100.72000122  97.40000153  98.49500275
 105.16500092 104.02999878 102.95500183 101.41999817 103.76999664
 101.95999908 133.73500061 130.73500061 125.26000214 121.89499664
 119.5        109.09500122 154.44500732 147.10499573 141.5
 135.43499756 116.625   

[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817  87.34500122  89.20999908  92.5
  91.31999969  92.10500336  92.51999664  92.48500061  92.95999908
  95.31999969  97.08999634  97.25499725  97.34999847  82.31500244
  82.91500092  86.77999878  90.76499939  92.23500061  92.14499664
  82.36499786  85.48999786  90.90499878  94.80999756  94.59999847
  93.84500122  81.62000275  83.00499725  86.51999664  86.95500183
  86.04499817  84.94499969  87.66000366  88.09999847  90.43499756
  90.36000061  90.06999969  86.05500031  90.49500275  93.45500183
  94.73500061  96.25        96.12999725  94.37000275  98.11000061
  98.76499939 101.23500061 100.72000122  97.40000153  98.49500275
 105.16500092 104.02999878 102.95500183 101.41999817 103.76999664
 101.95999908 133.73500061 130.73500061 125.26000214 121.89499664
 119.5        109.09500122 154.44500732 147.10499573 141.5
 135.43499756 116.625   

[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817  87.34500122  89.20999908  92.5
  91.31999969  92.10500336  92.51999664  92.48500061  92.95999908
  95.31999969  97.08999634  97.25499725  97.34999847  82.31500244
  82.91500092  86.77999878  90.76499939  92.23500061  92.14499664
  82.36499786  85.48999786  90.90499878  94.80999756  94.59999847
  93.84500122  81.62000275  83.00499725  86.51999664  86.95500183
  86.04499817  84.94499969  87.66000366  88.09999847  90.43499756
  90.36000061  90.06999969  86.05500031  90.49500275  93.45500183
  94.73500061  96.25        96.12999725  94.37000275  98.11000061
  98.76499939 101.23500061 100.72000122  97.40000153  98.49500275
 105.16500092 104.02999878 102.95500183 101.41999817 103.76999664
 101.95999908 133.73500061 130.73500061 125.26000214 121.89499664
 119.5        109.09500122 154.44500732 147.10499573 141.5
 135.43499756 116.625   

[110.45999908 109.65499878 106.55999756 104.48500061 102.06999969
  99.62000275  84.83000183  85.00499725  85.87999725  87.81500244
  88.625       86.79499817  87.34500122  89.20999908  92.5
  91.31999969  92.10500336  92.51999664  92.48500061  92.95999908
  95.31999969  97.08999634  97.25499725  97.34999847  82.31500244
  82.91500092  86.77999878  90.76499939  92.23500061  92.14499664
  82.36499786  85.48999786  90.90499878  94.80999756  94.59999847
  93.84500122  81.62000275  83.00499725  86.51999664  86.95500183
  86.04499817  84.94499969  87.66000366  88.09999847  90.43499756
  90.36000061  90.06999969  86.05500031  90.49500275  93.45500183
  94.73500061  96.25        96.12999725  94.37000275  98.11000061
  98.76499939 101.23500061 100.72000122  97.40000153  98.49500275
 105.16500092 104.02999878 102.95500183 101.41999817 103.76999664
 101.95999908 133.73500061 130.73500061 125.26000214 121.89499664
 119.5        109.09500122 154.44500732 147.10499573 141.5
 135.43499756 116.625   

...

[110.45999908 109.65499878 106.55999756 ...   9.55000019   9.35999966
   8.97000027]
(49878,)
------
8313
[110.45999908 109.65499878 106.55999756 ...   7.71500015   7.78499985
   7.56500006]
(49884,)
------
8314
[110.45999908 109.65499878 106.55999756 ...   9.49499989   9.57499981
   9.61999989]
(49890,)
------
8315
[110.45999908 109.65499878 106.55999756 ...  12.11999989  11.80500031
  11.56999969]
(49896,)
------
8316
[110.45999908 109.65499878 106.55999756 ...  12.05000019  12.47000027
  12.39000034]
(49902,)
------
8317
[110.45999908 109.65499878 106.55999756 ...   9.52999973   9.84500027
   9.85999966]
(49908,)
------
8318
[110.45999908 109.65499878 106.55999756 ...  20.27000046  19.03499985
  18.125     ]
(49914,)
------
8319
[110.45999908 109.65499878 106.55999756 ...  20.03499985  19.34000015
  19.06500053]
(49920,)
------
8320
[110.45999908 109.65499878 106.55999756 ...  14.77499962  14.90999985
  14.39000034]
(49926,)
------
8321
[110.45999908 109.65499878 106.55999756 ...  1

[110.45999908 109.65499878 106.55999756 ...   9.35000038   9.28999996
   9.28999996]
(50364,)
------
8394
[110.45999908 109.65499878 106.55999756 ...   8.77499962   9.36499977
   9.15999985]
(50370,)
------
8395
[110.45999908 109.65499878 106.55999756 ...  12.59000015  11.84500027
  11.5       ]
(50376,)
------
8396
[110.45999908 109.65499878 106.55999756 ...   9.57499981   9.70499992
   9.61499977]
(50382,)
------
8397
[110.45999908 109.65499878 106.55999756 ...   9.89500046   9.78499985
   9.51500034]
(50388,)
------
8398
[110.45999908 109.65499878 106.55999756 ...   8.57499981   8.56999969
   8.53499985]
(50394,)
------
8399
[110.45999908 109.65499878 106.55999756 ...  10.82499981  10.71500015
  10.52000046]
(50400,)
------
8400
[110.45999908 109.65499878 106.55999756 ...  11.65499973  11.55500031
  11.30500031]
(50406,)
------
8401
[110.45999908 109.65499878 106.55999756 ...   8.89500046   9.03999996
   9.27000046]
(50412,)
------
8402
[110.45999908 109.65499878 106.55999756 ...  1

[110.45999908 109.65499878 106.55999756 ...  94.61000061  85.46499634
  76.95500183]
(50832,)
------
8472
[110.45999908 109.65499878 106.55999756 ...  95.09999847  84.68499756
  76.01999664]
(50838,)
------
8473
[110.45999908 109.65499878 106.55999756 ... 106.87999725  95.30999756
  84.69999695]
(50844,)
------
8474
[110.45999908 109.65499878 106.55999756 ... 119.19499969 100.58999634
  92.06500244]
(50850,)
------
8475
[110.45999908 109.65499878 106.55999756 ... 107.875      100.80000305
  92.80999756]
(50856,)
------
8476
[110.45999908 109.65499878 106.55999756 ... 107.67500305  98.62999725
  89.71499634]
(50862,)
------
8477
[110.45999908 109.65499878 106.55999756 ... 100.55999756  89.67500305
  79.54499817]
(50868,)
------
8478
[110.45999908 109.65499878 106.55999756 ...  78.45500183  70.84500122
  63.43000031]
(50874,)
------
8479
[110.45999908 109.65499878 106.55999756 ...  57.89500046  49.17499924
  43.30500031]
(50880,)
------
8480
[110.45999908 109.65499878 106.55999756 ...  5

[110.45999908 109.65499878 106.55999756 ... 128.67999268 127.64499664
 124.69499969]
(51324,)
------
8554
[110.45999908 109.65499878 106.55999756 ... 121.63999939 120.
 118.37999725]
(51330,)
------
8555
[110.45999908 109.65499878 106.55999756 ... 124.94999695 124.02999878
 121.31500244]
(51336,)
------
8556
[110.45999908 109.65499878 106.55999756 ... 122.625      124.40000153
 126.34999847]
(51342,)
------
8557
[110.45999908 109.65499878 106.55999756 ... 132.58999634 132.95500183
 131.1000061 ]
(51348,)
------
8558
[110.45999908 109.65499878 106.55999756 ... 144.18499756 144.45500183
 148.11500549]
(51354,)
------
8559
[110.45999908 109.65499878 106.55999756 ... 157.61999512 157.45500183
 148.33000183]
(51360,)
------
8560
[110.45999908 109.65499878 106.55999756 ... 168.58000183 158.98500061
 155.02999878]
(51366,)
------
8561
[110.45999908 109.65499878 106.55999756 ... 187.36000061 181.13000488
 182.96000671]
(51372,)
------
8562
[110.45999908 109.65499878 106.55999756 ... 214.565002

[110.45999908 109.65499878 106.55999756 ...  38.83499908  45.125
  52.77999878]
(51810,)
------
8635
[110.45999908 109.65499878 106.55999756 ... 107.44499969 106.16500092
 101.51499939]
(51816,)
------
8636
[110.45999908 109.65499878 106.55999756 ... 101.50499725  99.48999786
  93.68499756]
(51822,)
------
8637
[110.45999908 109.65499878 106.55999756 ... 121.90000153 109.5
 104.48000336]
(51828,)
------
8638
[110.45999908 109.65499878 106.55999756 ... 114.69000244 104.73999786
  92.66999817]
(51834,)
------
8639
[110.45999908 109.65499878 106.55999756 ... 114.43000031  95.5
  82.90499878]
(51840,)
------
8640
[110.45999908 109.65499878 106.55999756 ...  97.625       82.11000061
  72.11499786]
(51846,)
------
8641
[110.45999908 109.65499878 106.55999756 ...  89.04000092  73.69499969
  62.39500046]
(51852,)
------
8642
[110.45999908 109.65499878 106.55999756 ...  48.97499847  44.18000031
  50.18000031]
(51858,)
------
8643
[110.45999908 109.65499878 106.55999756 ...  41.35499954  44.4399

In [18]:
test_ds_predict_q50_supervised = test_ds_predict_q50_all.reshape((-1, K))
test_ds_predict_q50_supervised.shape

(8683, 6)

In [19]:
test_ds_predict_q50_supervised[0:5]

array([[110.45999908, 109.65499878, 106.55999756, 104.48500061,
        102.06999969,  99.62000275],
       [ 84.83000183,  85.00499725,  85.87999725,  87.81500244,
         88.625     ,  86.79499817],
       [ 87.34500122,  89.20999908,  92.5       ,  91.31999969,
         92.10500336,  92.51999664],
       [ 92.48500061,  92.95999908,  95.31999969,  97.08999634,
         97.25499725,  97.34999847],
       [ 82.31500244,  82.91500092,  86.77999878,  90.76499939,
         92.23500061,  92.14499664]])

#### actual y

In [20]:
from helper import series_to_supervised, stage_series_to_supervised

In [21]:
test_df_supervised = series_to_supervised(test_df[['pollution']], n_hours, K)

In [22]:
test_df_supervised.iloc[:, -K:].shape

(8683, 6)

In [23]:
test_df_supervised_y = test_df_supervised.iloc[:, -K:]

In [24]:
test_df_supervised_y

,var1(t),var1(t+1),var1(t+2),var1(t+3),var1(t+4),var1(t+5)
35112,85.0,86.0,89.0,77.0,77.0,75.0
35113,86.0,89.0,77.0,77.0,75.0,80.0
35114,89.0,77.0,77.0,75.0,80.0,86.0
35115,77.0,77.0,75.0,80.0,86.0,95.0
35116,77.0,75.0,80.0,86.0,95.0,101.0
...,...,...,...,...,...,...
43790,9.0,11.0,8.0,9.0,10.0,8.0
43791,11.0,8.0,9.0,10.0,8.0,10.0
43792,8.0,9.0,10.0,8.0,10.0,10.0
43793,9.0,10.0,8.0,10.0,10.0,8.0


#### performance on test set

In [25]:
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae


print('MAE = {}'.format(float("{:.3f}".format(mae(test_ds_predict_q50_supervised, test_df_supervised_y)))))
print('RMSE = {}'.format(float("{:.3f}".format(np.sqrt(mse(test_ds_predict_q50_supervised, test_df_supervised_y))))))

MAE = 25.039
RMSE = 42.341
